# Geospatial visualization tutorial - 1

Hi all :-)

This is Yeonjun IN who made the kernel named as [\[DS4C\]time series geospatial eda using folium.](https://www.kaggle.com/mbnb8317/ds4c-time-series-geospatial-eda-using-folium)

This time, I made a Tutorial about Folium, a Package for Geospatial visualization, for novice of EDA.

I hope it helps

Thank you :)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import matplotlib.pyplot as plt

import folium 
from folium import plugins
# from folium.plugins import HeatMap

import json 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
patient = pd.read_csv('/kaggle/input/coronavirusdataset/PatientInfo.csv')
route = pd.read_csv('/kaggle/input/coronavirusdataset/PatientRoute.csv')
region = pd.read_csv('/kaggle/input/coronavirusdataset/Region.csv')

In [ ]:
patient.head()

In [ ]:
region.head()

# 1. plotting just a map 
제일 먼저, 그냥 지도 띄우기

```python
folium.Map([37,128]. zoom_start = 7, *tiles='Stamen Terrain'*)
folium.Map([37,128]. zoom_start = 7, *tiles='Stamen Toner'*)
```
There are several types of map, default, Stamen Terrain, and Stamen Toner. 

I prefer default option. 

In [ ]:
m = folium.Map([36, 128], zoom_start=7) 

# you can try these options
# m = folium.Map([37, 128], zoom_start=7, tiles='Stamen Terrain') 
# m = folium.Map([37, 128], zoom_start=7, tiles='Stamen Toner') 

m

### Additional tool : Fullscreen, MousePosition(get coordinates)
추가 기능 : 풀스크린으로 만들기, 마우스 갖다대면 좌표 뜨게 하기

In [ ]:
m = folium.Map([36, 128], zoom_start=7) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates.


m

# 2. MarkerCluster : Lets see the number of regional confirmed patient.
지역별로 몇명의 확진환자가 나왔는지 한번 봐볼까요?

In [ ]:
regional_patient = pd.merge(patient[['patient_id','confirmed_date','sex','age','province','city']],region[['province','city','latitude','longitude']], how = 'left', on = ['province','city'])
display(regional_patient.head())
print('There are a few missing values')
print('약간의 missing 값들이 있네요. 분석할 때 유의해야할 것 같습니다.')
display(regional_patient.isnull().sum() / regional_patient.shape[0])

### Captial Area(value of 724), including Seoul, Gyeonggi-do, Incheon etc., Gyeongsangbuk-do(972) look serious.

서울 경기 인천 지역, 즉 수도권 지역과, 경북지역의 감염자수가 가장 높은 것을 알 수 있습니다.

In [ ]:
regional_count = regional_patient[['latitude','longitude']].dropna() # drop missing values

m = folium.Map([36, 128], zoom_start=7)

plugins.MousePosition().add_to(m) 

plugins.MarkerCluster(regional_count).add_to(m)

m

# 3. Make subgroups to plot : Female / Male
여성, 남성 감염자를 하위그룹으로 하여 따로따로 볼 수 있도록 지도를 그려보겠습니다


In [ ]:
male_patient = regional_patient\
.query('sex in ("male")')[['latitude','longitude']].dropna()

female_patient = regional_patient\
.query('sex in ("female")')[['latitude','longitude']].dropna()


m = folium.Map([36, 128], zoom_start=7) 

initial = folium.plugins.MarkerCluster(control=False)

m.add_child(initial)


female_g = plugins.FeatureGroupSubGroup(initial, 'Female')
m.add_child(female_g)

male_g = plugins.FeatureGroupSubGroup(initial, 'Male') 
m.add_child(male_g)


plugins.MarkerCluster(female_patient).add_to(female_g)

plugins.MarkerCluster(male_patient).add_to(male_g)



folium.LayerControl(collapsed=False).add_to(m)

m

# 4. Make a Heat map : Instead of the number

지역별로 얼마나 많은 감염자가 있는지, 숫자로 보는 것이 아닌, heat map을 활용하여 시각적으로 나타내보겠습니다

**Heatmap parameter**

```python
folium.plugins.HeatMap(data, name=None, min_opacity=0.5, 
                       max_zoom=18, max_val=1.0, radius=25, blur=15, 
                       gradient=None, overlay=True, control=True, show=True, **kwargs)
```

```
- min_opacity (default 1.) – The minimum opacity the heat will start at.
- max_zoom (default 18) – Zoom level where the points reach maximum intensity (as intensity scales with zoom), equals maxZoom of the map by default
- max_val (float, default 1.) – Maximum point intensity
- radius (int, default 25) – Radius of each “point” of the heatmap
- blur (int, default 15) – Amount of blur
- gradient (dict, default None) – Color gradient config. e.g. {0.4: ‘blue’, 0.65: ‘lime’, 1: ‘red’}
- overlay (bool, default True) – Adds the layer as an optional overlay (True) or the base layer (False).
- control (bool, default True) – Whether the Layer will be included in LayerControls.
- show (bool, default True) – Whether the layer will be shown on opening (only for overlays).
```

[Link](https://python-visualization.github.io/folium/plugins.html)


In [ ]:
heat_data = regional_patient\
.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
.values\
.tolist()

m = folium.Map([36, 128], zoom_start=7)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m

### In my opinion, I prefer Heatmap to MarkCluster. How about you? :-)
저 개인적으로는 히트맵이 보기 좋네요 여러분들은 어떠신가요? ㅎㅎㅎ

### Additional try : MarkCluster Layer On the Heatmap Layer.

Sometimes, we need to plot several statistics on only one map.(Multivariate)

So i tried it with Heatmap and MarkCluster.

가끔 지도 위에 여러개의 통계량을 나타내야할 때가 있는데요, Heatmap과 MarkCluster로 시도해봤습니다


In [ ]:
heat_data = region[['latitude','longitude','nursing_home_count']].values.tolist() # nursing home count 

mark_data = regional_patient[['latitude','longitude']].dropna() # drop missing values

m = folium.Map([36, 128], zoom_start=7)

plugins.MarkerCluster(mark_data).add_to(m) # Layer 1

folium.plugins.HeatMap(heat_data, max_opacity = 0.3).add_to(m) # Layer 2

m

### Doesn't look good. 

보기 좋진 않네요.

# 5. Heatmap with time : time series heat map

```python
folium.plugins.HeatMapWithTime(data, index=None, name=None, 
                               radius=15, min_opacity=0, max_opacity=0.6, 
                               scale_radius=False, gradient=None, 
                               use_local_extrema=False, auto_play=False, 
                               display_index=True, index_steps=1, min_speed=0.1, 
                               max_speed=10, speed_step=0.1, position='bottomleft', 
                               overlay=True, control=True, show=True)
```

**Parameter** what i find important
```
data – The points you want to plot. The outer list corresponds to the various time steps in sequential order. (weight is in (0, 1] range and defaults to 1 if not specified for a point)

index  – the same length as data, or is replaced by a simple count if not specified.

radius (default 15.) – The radius used around points for the heatmap.

min_opacity (default 0) – The minimum opacity for the heatmap.

max_opacity (default 0.6) – The maximum opacity for the heatmap.

scale_radius (default False) – Scale the radius of the points based on the zoom level.

auto_play (default False) – Automatically play the animation across time.
```

[More](https://python-visualization.github.io/folium/plugins.html)

In [ ]:
#prepare data

regional_patient_bytime = regional_patient\
.groupby(['confirmed_date','latitude','longitude'])['patient_id'].count().reset_index()\
.sort_values(by = 'confirmed_date')

date_rng = pd.date_range(regional_patient_bytime['confirmed_date'].min(), regional_patient_bytime['confirmed_date'].max())
date_rng = [str(i)[:10] for i in date_rng]

all_date_region = pd.DataFrame({'confirmed_date' : np.repeat(date_rng,region[['latitude','longitude']].drop_duplicates().shape[0]),
              'latitude' : np.tile(list(region['latitude'].drop_duplicates()), len(date_rng)),
              'longitude' : np.tile(list(region['longitude'].drop_duplicates()), len(date_rng))})

all_date_region = pd.merge(all_date_region, regional_patient_bytime, on = ['confirmed_date','latitude','longitude'], how = 'left').fillna(0)
all_date_region['cumsum'] = all_date_region.groupby(['latitude','longitude'])['patient_id'].cumsum()

all_date_region = all_date_region[all_date_region['cumsum'] != 0]

time_index, data = [], []

for date in all_date_region['confirmed_date'].unique():
    time_index += [date[6:].replace('-','/')]
    temp_list = all_date_region.query('confirmed_date == @date')[['latitude','longitude','cumsum']].values.tolist()
    data += [temp_list]
    del temp_list

### HeatMapWithTime needs this type of input data.


In [ ]:
data[:5], time_index[:5]

In [ ]:
# make a map
    
m = folium.Map([36, 128], zoom_start=7)#, tiles = 'Mapbox Bright')

hm = folium.plugins.HeatMapWithTime(data, index=time_index, auto_play=False, min_opacity=0.3, radius = 25, )

hm.add_to(m)

for bottom, click in zip([60,90,120,150,180],['Loop','Forward','Play','Reverse','Backward']):
    
    # Thank you for html code. https://www.kaggle.com/poonsfc5/ds4c-covid-19-in-korea-eda-with-geo-data
    name = '''
            <div style="position: fixed; bottom: '''+str(bottom)+'''px; left: 50px; width: 100px; height: 29px; 
                        background-color: white; border:1px solid grey; z-index:9999; font-size:11px;text-align:center;"
                        >&nbsp; <br><b>Click : '''+ click +'''</b></div>'''  

    m.get_root().html.add_child(folium.Element(name))

m

# 6. Make Circles with timebar : Timestamped Geojson

```python
TimestampedGeoJson(data, transition_time=200, loop=True, auto_play=True, 
                   add_last_point=True, period='P1D', min_speed=0.1, max_speed=10, 
                   loop_button=False, date_options='YYYY-MM-DD HH:mm:ss', 
                   time_slider_drag_update=False, duration=None)
```

parameter i find important
```
period (str, default "P1D") –  ‘P1M’ 1/month, ‘P1D’ 1/day, ‘PT1H’ 1/hour, and ‘PT1M’ 1/minute
```

[more](https://python-visualization.github.io/folium/plugins.html)

In [ ]:
# prepare data
df_dropna = regional_patient.dropna()
df_dropna['timestamp'] = [i+'T00:00:00' for i in df_dropna['confirmed_date']]

points = []
for date in sorted(df_dropna['confirmed_date'].unique()):
    temp = df_dropna.query('confirmed_date == @date')
    
    temp_dict = {}
    temp_dict['coordinates'] = temp[['longitude','latitude']].values.tolist()
    temp_dict['dates'] = temp['timestamp'].values.tolist()
    
    points += [temp_dict]

features = [ { 'type': 'Feature', 
                  'geometry': { 'type': 'MultiPoint', 
                               'coordinates': point['coordinates'], }, 
                  'properties': { 'times': point['dates'],
                                 'icon' : 'circle'}
             } for point in points ] 

### TimestampedGeoJson needs this type of input data.

In [ ]:
features[:3]

In [ ]:
m = folium.Map( location=[36, 128], zoom_start=7 )

plugins.TimestampedGeoJson({ 
    'type': 'FeatureCollection', 
    'features': features, 
}, period='P1D', auto_play = False).add_to(m)

m

### Pydeck to be continued!

# Thank you for attention!!

